In [1]:
from xarray import DataArray

from weldx import Q_, U_, GenericSeries

## Allowed variables (expressions only)

In [2]:
class AllowedVarsSeries(GenericSeries):
    _allowed_variables = ["a"]

In [3]:
try:
    AllowedVarsSeries("a*b")
except ValueError as e:
    print(e)

'b' is not allowed as an expression variable of class AllowedVarsSeries


In [4]:
AllowedVarsSeries("4*a");

## Required variables (expressions only)

In [5]:
class RequiredVarsSeries(GenericSeries):
    _required_variables = ["c"]

In [6]:
try:
    RequiredVarsSeries("a*b")
except ValueError as e:
    print(e)

RequiredVarsSeries requires expression variable 'c'.


In [7]:
RequiredVarsSeries("a*c");

## Preprocess coordinates

In [8]:
def handle_time(instance, time, time_ref):
    from weldx import Time

    time = Time(time, time_ref)

    return dict(t=time.as_quantity())

In [9]:
class PreprocessCoordsSeries(GenericSeries):
    _evaluation_preprocessor = handle_time

### Discrete

In [10]:
pcs_discrete = PreprocessCoordsSeries(
    Q_([1, 2, 4], "m"), dims=["t"], coords={"t": Q_([2, 4, 7], "s")}
)

In [11]:
pcs_result_discrete = pcs_discrete(
    time="2000-01-01 15:00:05", time_ref="2000-01-01 15:00:00"
)
print(f"time: {pcs_result_discrete.data_array.t.data}")
print(f"data: {pcs_result_discrete.data}")

time: [5.]
data: [2.6666666666666665] m


In [12]:
try:
    pcs_discrete(x=3)
except TypeError:
    print("invalid input")

invalid input


### Expression

In [13]:
pcs_expr = PreprocessCoordsSeries("a * t", parameters=dict(a="3m/s"))

In [14]:
pcs_result_expr = pcs_expr(time="2000-01-01 15:00:05", time_ref="2000-01-01 15:00:00")
print(f"time: {pcs_result_expr.data_array.t.data}")
print(f"data: {pcs_result_expr.data}")

time: [5.]
data: [15.0] m


In [15]:
try:
    pcs_expr(x=3)
except TypeError:
    print("invalid input")

invalid input


## Required dimensions

In [16]:
class RequiredDimsSeries(GenericSeries):
    _required_dimensions = ["c"]

###  Discrete

In [17]:
try:
    RequiredDimsSeries(Q_([[1, 2], [3, 4]], "m"), ["a", "b"])
except ValueError as e:
    print(e)

RequiredDimsSeries requires dimension 'c'.


In [18]:
RequiredDimsSeries(Q_([[1, 2], [3, 4]], "m"), ["a", "c"]);

### Expression

In [19]:
try:
    RequiredDimsSeries("a*b")
except ValueError as e:
    print(e)

RequiredDimsSeries requires dimension 'c'.


In [20]:
RequiredDimsSeries("a*b", dims=dict(b="c"));

In [21]:
RequiredDimsSeries("a*c");

In [22]:
RequiredDimsSeries("a*b", parameters=dict(a=(Q_([1, 3], "m"), "c")));

## Required dimension units

In [23]:
class RequiredDimUnitsSeries(GenericSeries):
    _required_dimension_units = dict(t="s", c="m")

### Discrete

In [24]:
try:
    RequiredDimUnitsSeries(
        Q_([[1,2], [3,4]]), 
        dims=["t", "c"],
    )
except ValueError as e:
    print(e)

RequiredDimUnitsSeries requires dimension t to be have the unit dimensionality '[time]'


In [25]:
try:
    RequiredDimUnitsSeries(
        Q_([[1,2], [3,4]]), 
        dims=["t", "c"], 
        coords=dict(t=Q_([1, 2], "s"), c=Q_([1, 2], "K")),
    )
except ValueError as e:
    print(e)

RequiredDimUnitsSeries requires dimension c to be have the unit dimensionality '[length]'


In [26]:
RequiredDimUnitsSeries(
        Q_([[1,2], [3,4]]), 
        dims=["t", "c"], 
        coords=dict(t=Q_([1, 2], "s"), c=Q_([1, 2], "m")),
    );

### Expression

In [27]:
try:
    RequiredDimUnitsSeries("a*t + c")
except ValueError as e:
    print(e)

Expression can not be evaluated due to a unit dimensionality error. Ensure that the expressions parameters and the expected variable units are compatible. The original exception was:
Cannot convert from 'second' ([time]) to 'meter' ([length])


In [28]:
try:
    RequiredDimUnitsSeries(
        "a*t + c", 
        units=dict(t="K", c="m"),
        parameters=dict(a="m/K")
    )
except ValueError as e:
    print(e)

Expression can not be evaluated due to a unit dimensionality error. Ensure that the expressions parameters and the expected variable units are compatible. The original exception was:
Cannot convert from 'meter * second / kelvin' ([length] * [time] / [temperature]) to 'meter' ([length])


In [29]:
try:
    RequiredDimUnitsSeries(
        "a*t + c", 
        units=dict(t="s"),
        parameters=dict(a="K/s", c="5K")
    )
except ValueError as e:
    print(e)

KeyError: 'c'

In [ ]:
RequiredDimUnitsSeries(
        "a*t + c", 
        units=dict(t="s", c="m"),
        parameters=dict(a="m/s")
    );

In [ ]:
RequiredDimUnitsSeries(
        "a*t + c", 
        units=dict(t="s"),
        parameters=dict(a="m/s", c="m")
    );

In [ ]:
RequiredDimUnitsSeries(
        "a*t + c", 
        units=dict(a="m/s"),
        parameters=dict(t="s", c="5m")
    );

## Required dimension coordinates

In [ ]:
class RequiredDimCoordsSeries(GenericSeries):
    _required_dimension_coordinates = dict(c=["x", "y"])

### Discrete

In [ ]:
try:
    RequiredDimCoordsSeries(
        Q_([[1,2], [3,4]]), 
        dims=["t", "c"],
    )
except ValueError as e:
    print(e)

In [ ]:
try:
    RequiredDimCoordsSeries(
        Q_([[1,2], [3,4]]), 
        dims=["t", "c"],
        coords=dict(c=["a", "b"])
    )
except ValueError as e:
    print(e)

In [ ]:
RequiredDimCoordsSeries(
    Q_([[1,2], [3,4]]), 
    dims=["t", "c"],
    coords=dict(c=["x", "y"])
);

### Expression

In [ ]:
try:
    RequiredDimCoordsSeries(
        "a*t + c"
    )
except ValueError as e:
    print(e)

In [ ]:
b = DataArray(Q_([1,2],""), dims=["c"])

try:
    RequiredDimCoordsSeries(
        "a*t + b", 
        parameters=dict(b=b)
    )
except ValueError as e:
    print(e)

In [ ]:
b = DataArray(Q_([1,2],""), dims=["c"], coords=dict(c=["a", "b"]))
try:
    RequiredDimCoordsSeries(
        "a*t + b", 
        parameters=dict(b=b)
    )
except ValueError as e:
    print(e)

In [ ]:
b = DataArray(Q_([1,2],""), dims=["c"], coords=dict(c=["x", "y"]))
RequiredDimCoordsSeries(
    "a*t + b", 
    parameters=dict(b=b)
);

## Required unit dimensionality

In [ ]:
class RequiredUnitDimSeries(GenericSeries):
    _required_unit_dimensionality = U_("m").dimensionality

### Discrete

In [ ]:
try:
    RequiredUnitDimSeries(Q_("2K"))
except ValueError as e:
    print(e)

In [ ]:
RequiredUnitDimSeries(Q_("2m"));

### Expression

In [ ]:
try:
    RequiredUnitDimSeries("a*b", units=dict(a="K"))
except ValueError as e:
    print(e)

In [ ]:
RequiredUnitDimSeries("a*b", units=dict(a="m"));

In [ ]:
coords = DataArray(Q_([1, 2, 3], "s"), dims=["t"]).pint.dequantify().pint.dequantify()
data = DataArray(Q_([1, 2, 3], "m"), dims=["t"], coords={"t": coords})
print(data.coords["t"])
print(coords)